# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import joblib
torch.manual_seed(42)
np.random.seed(42)
warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# Explore Data


In [2]:
# main datasets
health_path = "/kaggle/input/dapdatasetnew/DAP_2022_HLES_health_condition_v1.0.csv"
overview_path = "/kaggle/input/dapdatasetnew/DAP_2022_DogOverview_v1.0.csv"
codebook_path = "/kaggle/input/dapdatasetnew/DAP_2023_CODEBOOK_v1.1.csv"

#  data with proper data types
df_health = pd.read_csv(health_path, dtype={'hs_condition': str, 'hs_condition_other_description': str})
df_overview = pd.read_csv(overview_path)
df_codebook = pd.read_csv(codebook_path)

print("Dataset shapes:")
print(f"Health conditions: {df_health.shape}")
print(f"Dog overview: {df_overview.shape}")
print(f"Codebook: {df_codebook.shape}")

Dataset shapes:
Health conditions: (119271, 12)
Dog overview: (43517, 35)
Codebook: (2908, 5)


# Data Preprocessing


In [3]:
# disease code mapping from codebook
condition_map_row = df_codebook[
    (df_codebook['DataFile'].str.contains('health_condition')) & 
    (df_codebook['Variable'] == 'hs_condition')
].iloc[0]

# mapping dictionary
code_to_disease = {}
values = condition_map_row['Values'].split(' | ')
labels = condition_map_row['ValueLabels'].split(' | ')

for val, label in zip(values, labels):
    code = val.split('.')[0]  
    code_to_disease[code] = label

print(f"Found {len(code_to_disease)} disease codes")

# map condition to disease names
df_health['hs_condition'] = df_health['hs_condition'].str.split('.').str[0]
df_health['disease_name'] = df_health['hs_condition'].map(code_to_disease)

# filter rows without valid disease 
df_health = df_health.dropna(subset=['disease_name'])
print(f"After mapping: {df_health.shape[0]} records with valid disease names")

demographic_cols = ['dog_id', 'Breed', 'Sex_Class_at_HLES', 'Estimated_Age_Years_at_HLES']
df_demo = df_overview[demographic_cols].copy()

# Merge health data with demographics
df_merged = df_health.merge(df_demo, on='dog_id', how='left')

print(f"Merged dataset shape: {df_merged.shape}")

Found 329 disease codes
After mapping: 118505 records with valid disease names
Merged dataset shape: (118505, 16)


# Symptom Text Generation


In [4]:
def create_structured_symptom_text(row):    
    # get the actual disease name
    disease_name = row['disease_name']
    
    symptoms = []
    
    # symptoms based on disease type
    if 'dental' in disease_name.lower() or 'teeth' in disease_name.lower() or 'gingivitis' in disease_name.lower():
        symptoms.extend(["bad breath", "difficulty eating", "yellow teeth", "red gums", "dropping food"])
    elif 'ear infection' in disease_name.lower() or 'otitis' in disease_name.lower():
        symptoms.extend(["head shaking", "ear scratching", "ear odor", "red ears", "ear discharge"])
    elif 'giardia' in disease_name.lower() or 'parasite' in disease_name.lower():
        symptoms.extend(["watery diarrhea", "vomiting", "weight loss", "lethargy", "dehydration"])
    elif 'allerg' in disease_name.lower() or 'pruritis' in disease_name.lower() or 'dermatitis' in disease_name.lower():
        symptoms.extend(["itching", "skin redness", "excessive licking", "hot spots", "hair loss"])
    elif 'arthritis' in disease_name.lower() or 'joint' in disease_name.lower():
        symptoms.extend(["limping", "stiffness", "difficulty rising", "joint pain", "reluctance to jump"])
    elif 'kennel cough' in disease_name.lower() or 'cough' in disease_name.lower():
        symptoms.extend(["harsh cough", "gagging", "nasal discharge", "lethargy", "fever"])
    elif 'bite' in disease_name.lower() or 'trauma' in disease_name.lower() or 'laceration' in disease_name.lower():
        symptoms.extend(["visible wound", "pain", "swelling", "bleeding", "limping"])
    elif 'cataract' in disease_name.lower():
        symptoms.extend(["cloudy eyes", "bumping into objects", "vision impairment", "eye redness"])
    elif 'murmur' in disease_name.lower() or 'heart' in disease_name.lower():
        symptoms.extend(["exercise intolerance", "coughing", "weakness", "breathing difficulty", "collapse"])
    else:
        # generic symptoms for other conditions
        symptoms.extend(["lethargy", "reduced appetite", "behavior changes", "fever"])
    
    if pd.notna(row['hs_condition_other_description']) and len(str(row['hs_condition_other_description'])) > 10:
        primary_symptoms = str(row['hs_condition_other_description'])
    else:
        np.random.shuffle(symptoms)
        primary_symptoms = ", ".join(symptoms[:min(4, len(symptoms))])
    
    # build the text
    text_parts = [
        f"Symptoms: {primary_symptoms}",
        f"Duration: {np.random.choice(['acute', 'chronic', 'recurrent'])}"
    ]
    
    # severity context
    severity = np.random.choice(['mild', 'moderate', 'severe'])
    text_parts.append(f"Severity: {severity}")
    
    # demographics
    demo = []
    if pd.notna(row['Breed']):
        demo.append(row['Breed'])
    if pd.notna(row['Sex_Class_at_HLES']):
        demo.append(row['Sex_Class_at_HLES'])
    if pd.notna(row['Estimated_Age_Years_at_HLES']):
        demo.append(f"{int(row['Estimated_Age_Years_at_HLES'])} years")
    
    if demo:
        text_parts.append(f"Patient: {', '.join(demo)}")
    
    return ". ".join(text_parts)
    
# structured symptom 
df_merged['symptom_text'] = df_merged.apply(create_structured_symptom_text, axis=1)

# filter top diseases 
top_diseases = df_merged['disease_name'].value_counts().head(20).index.tolist()
df_filtered = df_merged[df_merged['disease_name'].isin(top_diseases)].copy()

print(f"Clean dataset: {df_filtered.shape[0]} records")
print(f"Top diseases: {len(top_diseases)} classes")

Clean dataset: 51841 records
Top diseases: 20 classes


# Treatment System


In [5]:
TREATMENT_SUGGESTIONS = {
    'dental': ["Dental cleaning under anesthesia", "Tooth extraction if needed", "Antibiotics for infection", 
               "Pain management medication", "Dental diet recommendations"],
    'ear infection': ["Ear cleaning solution", "Topical antibiotics", "Oral antibiotics if severe", 
                     "Anti-inflammatory medication", "Ear mite treatment if applicable"],
    'giardia': ["Metronidazole or fenbendazole course", "Supportive care for dehydration", 
               "Environmental decontamination", "Probiotics for gut health", "Bland diet recommendation"],
    'allergy': ["Antihistamines (diphenhydramine)", "Steroids if severe inflammation", "Allergy testing recommendation",
               "Immunotherapy options", "Dietary changes", "Medicated shampoos"],
    'arthritis': ["NSAIDs for pain management", "Joint supplements (glucosamine)", "Weight management plan",
                 "Physical therapy exercises", "Laser therapy options", "Orthopedic bedding"],
    'kennel cough': ["Cough suppressants", "Antibiotics if bacterial infection", "Rest and isolation",
                    "Humidifier for comfort", "Nutritional support"],
    'trauma': ["Wound cleaning and closure", "Pain management", "Antibiotics for infection prevention",
              "Rest and restricted activity", "Follow-up monitoring"],
    'cataract': ["Surgical consultation", "Management of underlying causes", "Supportive care for vision impairment",
                "Environmental adaptations"],
    'heart murmur': ["Cardiac ultrasound recommended", "Medication to support heart function", "Restricted exercise",
                    "Regular monitoring", "Specialized cardiac diet"],
    'default': ["Veterinary consultation recommended", "Symptomatic treatment", "Monitoring of symptoms",
               "Follow-up care", "Diagnostic testing if persistent"]
}

def get_treatment_suggestions(disease_name):
    disease_lower = disease_name.lower()
    
    if any(keyword in disease_lower for keyword in ['dental', 'teeth', 'gingivitis', 'periodontal']):
        return TREATMENT_SUGGESTIONS['dental']
    elif any(keyword in disease_lower for keyword in ['ear infection', 'otitis', 'ear mites']):
        return TREATMENT_SUGGESTIONS['ear infection']
    elif any(keyword in disease_lower for keyword in ['giardia', 'parasite', 'intestinal worms']):
        return TREATMENT_SUGGESTIONS['giardia']
    elif any(keyword in disease_lower for keyword in ['allerg', 'pruritis', 'dermatitis', 'skin allergy']):
        return TREATMENT_SUGGESTIONS['allergy']
    elif any(keyword in disease_lower for keyword in ['arthritis', 'joint', 'orthopedic', 'hip dysplasia']):
        return TREATMENT_SUGGESTIONS['arthritis']
    elif any(keyword in disease_lower for keyword in ['kennel cough', 'tracheobronchitis', 'respiratory infection']):
        return TREATMENT_SUGGESTIONS['kennel cough']
    elif any(keyword in disease_lower for keyword in ['bite', 'trauma', 'laceration', 'wound', 'injury']):
        return TREATMENT_SUGGESTIONS['trauma']
    elif any(keyword in disease_lower for keyword in ['cataract']):
        return TREATMENT_SUGGESTIONS['cataract']
    elif any(keyword in disease_lower for keyword in ['murmur', 'cardiac', 'heart disease']):
        return TREATMENT_SUGGESTIONS['heart murmur']
    else:
        return TREATMENT_SUGGESTIONS['default']

# Severity System


In [6]:
SEVERITY_LEVELS = {
    'mild': 1,
    'moderate': 2, 
    'severe': 3,
    'critical': 4
}

def calculate_symptom_severity(symptoms_text):
    symptom_text_lower = symptoms_text.lower()
    severity_score = 1  
    
    # critical indicators 
    critical_indicators = [
        'unconscious', 'seizure', 'paralysis', 'collapse', 
        'pale gums', 'bloat', 'distended abdomen', 'difficulty breathing'
    ]
    
    # severe indicators 
    severe_indicators = [
        'bleeding', 'vomiting blood', 'bloody diarrhea', 'unable to stand',
        'crying in pain', 'swollen abdomen', 'high fever'
    ]
    
    # moderate indicators
    moderate_indicators = [
        'vomiting', 'diarrhea', 'lethargy', 'pain', 'limp', 'not eating',
        'fever', 'coughing', 'whining', 'difficulty urinating'
    ]
    
    # check for critical indicators
    for indicator in critical_indicators:
        if indicator in symptom_text_lower:
            severity_score = max(severity_score, SEVERITY_LEVELS['critical'])
    
    # check for severe indicators
    for indicator in severe_indicators:
        if indicator in symptom_text_lower:
            severity_score = max(severity_score, SEVERITY_LEVELS['severe'])
    
    # check for moderate indicators
    for indicator in moderate_indicators:
        if indicator in symptom_text_lower:
            severity_score = max(severity_score, SEVERITY_LEVELS['moderate'])
    
    return severity_score

# Data Preparation


In [7]:
# encode disease labels
label_encoder = LabelEncoder()
df_filtered['disease_label'] = label_encoder.fit_transform(df_filtered['disease_name'])

unique_dogs = df_filtered['dog_id'].unique()
train_dogs, temp_dogs = train_test_split(unique_dogs, test_size=0.3, random_state=42)
val_dogs, test_dogs = train_test_split(temp_dogs, test_size=0.5, random_state=42)

# create splits
train_df = df_filtered[df_filtered['dog_id'].isin(train_dogs)]
val_df = df_filtered[df_filtered['dog_id'].isin(val_dogs)]
test_df = df_filtered[df_filtered['dog_id'].isin(test_dogs)]

print(f"Train: {train_df.shape[0]} records, {len(train_dogs)} dogs")
print(f"Val: {val_df.shape[0]} records, {len(val_dogs)} dogs")
print(f"Test: {test_df.shape[0]} records, {len(test_dogs)} dogs")

Train: 36401 records, 17950 dogs
Val: 7651 records, 3847 dogs
Test: 7789 records, 3847 dogs


# Dataset and DataLoader Setup


In [8]:
class DogDiseaseDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# create datasets
train_dataset = DogDiseaseDataset(
    train_df['symptom_text'].tolist(),
    train_df['disease_label'].tolist(),
    tokenizer
)

val_dataset = DogDiseaseDataset(
    val_df['symptom_text'].tolist(),
    val_df['disease_label'].tolist(),
    tokenizer
)

test_dataset = DogDiseaseDataset(
    test_df['symptom_text'].tolist(),
    test_df['disease_label'].tolist(),
    tokenizer
)

# create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

Training batches: 2276
Validation batches: 479
Test batches: 487


# Model Architecture


In [13]:
class EnhancedDiseaseClassifier(nn.Module):
    def __init__(self, num_classes, model_name="emilyalsentzer/Bio_ClinicalBERT"):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        # hidden layer
        self.hidden = nn.Linear(self.bert.config.hidden_size, 256)
        self.classifier = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        
        # hidden layer
        hidden = self.relu(self.hidden(pooled_output))
        hidden = self.dropout(hidden)
        
        logits = self.classifier(hidden)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()  
            loss = loss_fct(logits, labels)
        
        return loss, logits

# initialize model
num_classes = len(label_encoder.classes_)
model = EnhancedDiseaseClassifier(num_classes=num_classes)
model = model.to(device)

print(f"Model initialized for {num_classes} disease classes")

Model initialized for 20 disease classes


# Training Setup


In [14]:
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)  

# parameters
epochs = 15
total_steps = len(train_loader) * epochs

# learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0, 
    num_training_steps=total_steps
)

print("Training setup completed")

Training setup completed


# Training Functions


In [15]:
def train_epoch(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        
        # forward pass
        loss, logits = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        
        # backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        
        # accuracy
        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions.double() / len(data_loader.dataset)
    
    return avg_loss, accuracy

def eval_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            loss, logits = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            
            total_loss += loss.item()
            
            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions.double() / len(data_loader.dataset)
    
    return avg_loss, accuracy, all_preds, all_labels

# Model Training


In [16]:
# training execution
best_accuracy = 0
best_val_loss = float('inf')
patience = 5  # Increased patience
patience_counter = 0
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

print("Starting model training.")
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    print("-" * 50)
    
    # train
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    
    # validate
    val_loss, val_acc, val_preds, val_labels = eval_model(model, val_loader, device)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    # Early stopping and model saving logic
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print("Saved best model")
    else:
        patience_counter += 1
        print(f"No improvement for {patience_counter} epoch(s)")
    
    # check for early stopping
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs!")
        break

print("\nTraining completed")

# convert accuracy tensors for plotting
train_accuracies_cpu = [acc.cpu().item() for acc in train_accuracies]
val_accuracies_cpu = [acc.cpu().item() for acc in val_accuracies]

# plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies_cpu, label='Training Accuracy')
plt.plot(val_accuracies_cpu, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig('training_history_enhanced.png')
plt.show()

Starting model training.

Epoch 1/15
--------------------------------------------------
Train Loss: 0.9918, Train Acc: 0.5989
Val Loss: 0.7865, Val Acc: 0.6229
Saved best model

Epoch 2/15
--------------------------------------------------
Train Loss: 0.8057, Train Acc: 0.6189
Val Loss: 0.7744, Val Acc: 0.6330
Saved best model

Epoch 3/15
--------------------------------------------------
Train Loss: 0.7867, Train Acc: 0.6253
Val Loss: 0.7692, Val Acc: 0.6368
Saved best model

Epoch 4/15
--------------------------------------------------
Train Loss: 0.7761, Train Acc: 0.6305
Val Loss: 0.7642, Val Acc: 0.6373
Saved best model

Epoch 5/15
--------------------------------------------------
Train Loss: 0.7670, Train Acc: 0.6345
Val Loss: 0.7634, Val Acc: 0.6393
Saved best model

Epoch 6/15
--------------------------------------------------
Train Loss: 0.7601, Train Acc: 0.6385
Val Loss: 0.7632, Val Acc: 0.6376
No improvement for 1 epoch(s)

Epoch 7/15
--------------------------------------

# Model Evaluation


In [38]:
# save the base BERT model locally
bert_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
bert_model.save_pretrained('model_assets/bio_clinical_bert')
print("Base BERT model saved locally")

# load the best model
model.load_state_dict(torch.load('best_model.pth'))

# test evaluation
test_loss, test_acc, test_preds, test_labels = eval_model(model, test_loader, device)
print(f"\nTest Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.1f}%)")

Base BERT model saved locally

Test Results:
Test Loss: 0.7769
Test Accuracy: 0.6378 (63.8%)


# Prediction Functions


In [32]:
def predict_with_confidence(model, tokenizer, symptom_text, top_k=3):
    encoding = tokenizer(
        symptom_text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    model.eval()
    with torch.no_grad():
        _, logits = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(logits, dim=1)
        top_probs, top_indices = torch.topk(probabilities, top_k)
    
    results = []
    for i in range(top_k):
        disease = label_encoder.inverse_transform([top_indices[0][i].item()])[0]
        prob = top_probs[0][i].item()
        
        # generate confidence explanation
        if prob > 0.7:
            confidence_level = "High confidence"
            explanation = "Clear symptom patterns match this condition"
        elif prob > 0.5:
            confidence_level = "Moderate confidence" 
            explanation = "Good symptom alignment with some uncertainty"
        elif prob > 0.3:
            confidence_level = "Low confidence"
            explanation = "Symptoms could indicate multiple conditions"
        else:
            confidence_level = "Very low confidence"
            explanation = "Limited symptom information available"
        
        results.append({
            'disease': disease, 
            'confidence': prob,
            'confidence_level': confidence_level,
            'explanation': explanation,
            'treatments': get_treatment_suggestions(disease),
            'severity': calculate_symptom_severity(symptom_text)
        })
    
    return results

# Deployment Function with Hybrid Approach


In [33]:
def diagnose_dog(symptom_text, breed=None, age=None, sex=None, top_k=3):
    # complete clinical description
    clinical_text = symptom_text
    if breed:
        clinical_text += f" Breed: {breed}"
    if age:
        clinical_text += f" Age: {age} years"
    if sex:
        clinical_text += f" Sex: {sex}"
    
    # predictions with confidence
    predictions = predict_with_confidence(model, tokenizer, clinical_text, top_k)
    top_disease = predictions[0]['disease']
    
    # calculate severity
    severity_score = calculate_symptom_severity(clinical_text)
    severity_level = list(SEVERITY_LEVELS.keys())[list(SEVERITY_LEVELS.values()).index(severity_score)]
    
    # format comprehensive results
    result = {
        'symptoms': symptom_text,
        'demographics': {'breed': breed, 'age': age, 'sex': sex},
        'severity': {'score': severity_score, 'level': severity_level},
        'urgent_care': severity_score >= SEVERITY_LEVELS['severe'],
        'predictions': predictions,
        'top_disease': top_disease,
        'top_confidence': predictions[0]['confidence'],
        'top_treatments': predictions[0]['treatments'][:3]  
    }
    
    return result

# Saving the model

In [ ]:
# directory for all model assets
import os
import json
import joblib
from collections import OrderedDict

os.makedirs('model_assets', exist_ok=True)

# save the trained model weights
torch.save(model.state_dict(), 'model_assets/dog_disease_model.pth')

# save the label encoder
joblib.dump(label_encoder, 'model_assets/label_encoder.pkl')

# save the tokenizer
tokenizer.save_pretrained('model_assets/tokenizer')

# save the code to disease mapping
with open('model_assets/code_to_disease.json', 'w') as f:
    json.dump(code_to_disease, f)

# save the treatment suggestions
with open('model_assets/treatment_suggestions.json', 'w') as f:
    json.dump(TREATMENT_SUGGESTIONS, f)

# save the severity levels
with open('model_assets/severity_levels.json', 'w') as f:
    json.dump(SEVERITY_LEVELS, f)

# save the class names
class_names = label_encoder.classes_.tolist()
with open('model_assets/class_names.json', 'w') as f:
    json.dump(class_names, f)

# save model configuration
model_config = {
    "model_name": "emilyalsentzer/Bio_ClinicalBERT",
    "num_classes": num_classes,
    "max_length": 128,
    "device": str(device)
}

with open('model_assets/model_config.json', 'w') as f:
    json.dump(model_config, f)

# complete inference script the web app
inference_script = '''
import torch
import joblib
import json
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import LabelEncoder
import re

class DogDiseaseClassifier:
    def __init__(self, model_assets_path):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model_assets_path = model_assets_path
        
        # load configuration
        with open(f'{model_assets_path}/model_config.json', 'r') as f:
            self.config = json.load(f)
        
        # load label encoder
        self.label_encoder = joblib.load(f'{model_assets_path}/label_encoder.pkl')
        
        # load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(f'{model_assets_path}/tokenizer')
        
        # load treatment suggestions
        with open(f'{model_assets_path}/treatment_suggestions.json', 'r') as f:
            self.treatment_suggestions = json.load(f)
            
        # load severity levels
        with open(f'{model_assets_path}/severity_levels.json', 'r') as f:
            self.severity_levels = json.load(f)
        
        # initialize model
        self.model = self._create_model()
        self.model.load_state_dict(torch.load(f'{model_assets_path}/dog_disease_model.pth', 
                                            map_location=self.device))
        self.model.to(self.device)
        self.model.eval()
    
    def _create_model(self):
        class EnhancedDiseaseClassifier(torch.nn.Module):
            def __init__(self, num_classes, model_name="emilyalsentzer/Bio_ClinicalBERT"):
                super().__init__()
                self.bert = AutoModel.from_pretrained(f'{self.model_assets_path}/bio_clinical_bert', 
                                                     local_files_only=True)                
                self.dropout = torch.nn.Dropout(0.3)
                self.hidden = torch.nn.Linear(self.bert.config.hidden_size, 256)
                self.classifier = torch.nn.Linear(256, num_classes)
                self.relu = torch.nn.ReLU()
            
            def forward(self, input_ids, attention_mask, labels=None):
                outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
                pooled_output = outputs.last_hidden_state[:, 0, :]
                pooled_output = self.dropout(pooled_output)
                hidden = self.relu(self.hidden(pooled_output))
                hidden = self.dropout(hidden)
                logits = self.classifier(hidden)
                
                loss = None
                if labels is not None:
                    loss_fct = torch.nn.CrossEntropyLoss()  
                    loss = loss_fct(logits, labels)
                
                return loss, logits
                
        return EnhancedDiseaseClassifier(self.config["num_classes"], self.config["model_name"])
    
    def get_treatment_suggestions(self, disease_name):
        disease_lower = disease_name.lower()
        
        if any(keyword in disease_lower for keyword in ['dental', 'teeth', 'gingivitis', 'periodontal']):
            return self.treatment_suggestions['dental']
        elif any(keyword in disease_lower for keyword in ['ear infection', 'otitis', 'ear mites']):
            return self.treatment_suggestions['ear infection']
        elif any(keyword in disease_lower for keyword in ['giardia', 'parasite', 'intestinal worms']):
            return self.treatment_suggestions['giardia']
        elif any(keyword in disease_lower for keyword in ['allerg', 'pruritis', 'dermatitis', 'skin allergy']):
            return self.treatment_suggestions['allergy']
        elif any(keyword in disease_lower for keyword in ['arthritis', 'joint', 'orthopedic', 'hip dysplasia']):
            return self.treatment_suggestions['arthritis']
        elif any(keyword in disease_lower for keyword in ['kennel cough', 'tracheobronchitis', 'respiratory infection']):
            return self.treatment_suggestions['kennel cough']
        elif any(keyword in disease_lower for keyword in ['bite', 'trauma', 'laceration', 'wound', 'injury']):
            return self.treatment_suggestions['trauma']
        elif any(keyword in disease_lower for keyword in ['cataract']):
            return self.treatment_suggestions['cataract']
        elif any(keyword in disease_lower for keyword in ['murmur', 'cardiac', 'heart disease']):
            return self.treatment_suggestions['heart murmur']
        else:
            return self.treatment_suggestions['default']
    
    def calculate_symptom_severity(self, symptoms_text):
        symptom_text_lower = symptoms_text.lower()
        severity_score = 1
        
        # critical indicators 
        critical_indicators = [
            'unconscious', 'seizure', 'paralysis', 'collapse', 
            'pale gums', 'bloat', 'distended abdomen', 'difficulty breathing'
        ]
        
        # severe indicators 
        severe_indicators = [
            'bleeding', 'vomiting blood', 'bloody diarrhea', 'unable to stand',
            'crying in pain', 'swollen abdomen', 'high fever'
        ]
        
        # moderate indicators
        moderate_indicators = [
            'vomiting', 'diarrhea', 'lethargy', 'pain', 'limp', 'not eating',
            'fever', 'coughing', 'whining', 'difficulty urinating'
        ]
        
        # check for critical indicators
        for indicator in critical_indicators:
            if re.search(r'\b' + re.escape(indicator) + r'\b', symptom_text_lower):
                severity_score = max(severity_score, self.severity_levels['critical'])
        
        # check for severe indicators
        for indicator in severe_indicators:
            if re.search(r'\b' + re.escape(indicator) + r'\b', symptom_text_lower):
                severity_score = max(severity_score, self.severity_levels['severe'])
        
        # check for moderate indicators
        for indicator in moderate_indicators:
            if re.search(r'\b' + re.escape(indicator) + r'\b', symptom_text_lower):
                severity_score = max(severity_score, self.severity_levels['moderate'])
        
        return severity_score
    
    def predict(self, symptom_text, breed=None, age=None, sex=None, top_k=3):
        # validate top_k parameter
        if not isinstance(top_k, int) or top_k <= 0:
            top_k = 3  
        top_k = min(top_k, self.config["num_classes"])
        
        # clinical description
        clinical_text = symptom_text
        if breed:
            clinical_text += f" Breed: {breed}"
        if age:
            clinical_text += f" Age: {age} years"
        if sex:
            clinical_text += f" Sex: {sex}"
        
        # tokenize input
        encoding = self.tokenizer(
            clinical_text,
            truncation=True,
            padding='max_length',
            max_length=self.config["max_length"],
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        
        # get prediction
        with torch.no_grad():
            _, logits = self.model(input_ids=input_ids, attention_mask=attention_mask)
            probabilities = torch.softmax(logits, dim=1)
            top_probs, top_indices = torch.topk(probabilities, top_k)
        
        results = []
        for i in range(top_k):
            disease = self.label_encoder.inverse_transform([top_indices[0][i].item()])[0]
            prob = top_probs[0][i].item()
            
            # generate confidence explanation
            if prob > 0.7:
                confidence_level = "High confidence"
                explanation = "Clear symptom patterns match this condition"
            elif prob > 0.5:
                confidence_level = "Moderate confidence" 
                explanation = "Good symptom alignment with some uncertainty"
            elif prob > 0.3:
                confidence_level = "Low confidence"
                explanation = "Symptoms could indicate multiple conditions"
            else:
                confidence_level = "Very low confidence"
                explanation = "Limited symptom information available"
            
            results.append({
                'disease': disease, 
                'confidence': prob,
                'confidence_level': confidence_level,
                'explanation': explanation,
                'treatments': self.get_treatment_suggestions(disease),
                'severity': self.calculate_symptom_severity(symptom_text)
            })
        
        # calculate severity
        severity_score = self.calculate_symptom_severity(clinical_text)
        score_to_level = {v: k for k, v in self.severity_levels.items()}
        severity_level = score_to_level.get(severity_score, "unknown")        
       
        # format comprehensive results
        result = {
            'symptoms': symptom_text,
            'demographics': {'breed': breed, 'age': age, 'sex': sex},
            'severity': {'score': severity_score, 'level': severity_level},
            'urgent_care': severity_score >= self.severity_levels['severe'],
            'predictions': results,
            'top_disease': results[0]['disease'],
            'top_confidence': results[0]['confidence'],
            'top_treatments': results[0]['treatments'][:3]  
        }
        
        return result

'''

with open('model_assets/inference.py', 'w') as f:
    f.write(inference_script)

print("All model assets saved successfully!")
print("Files created:")
for root, dirs, files in os.walk('model_assets'):
    for file in files:
        print(f"  - {os.path.join(root, file)}")

All model assets saved successfully!
Files created:
  - model_assets/class_names.json
  - model_assets/inference.py
  - model_assets/treatment_suggestions.json
  - model_assets/severity_levels.json
  - model_assets/label_encoder.pkl
  - model_assets/code_to_disease.json
  - model_assets/dog_disease_model.pth
  - model_assets/model_config.json
  - model_assets/bio_clinical_bert/model.safetensors
  - model_assets/bio_clinical_bert/config.json
  - model_assets/tokenizer/vocab.txt
  - model_assets/tokenizer/special_tokens_map.json
  - model_assets/tokenizer/tokenizer.json
  - model_assets/tokenizer/tokenizer_config.json


In [40]:
# requirements.txt for the web app
requirements = '''
torch>=1.9.0
transformers>=4.15.0
scikit-learn>=0.24.2
pandas>=1.3.0
numpy>=1.21.0
joblib>=1.0.0
flask>=2.0.0
fastapi>=0.68.0
uvicorn>=0.15.0
python-multipart>=0.0.5
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print("requirements.txt created")

requirements.txt created


In [51]:
# zip archive of model assets
import shutil
from IPython.display import FileLink, display
import os

shutil.make_archive('/kaggle/working/model_assets', 'zip', '/kaggle/working/model_assets')

# zip of the essential files
essential_files = [
    '/kaggle/working/requirements.txt',
    '/kaggle/working/model_assets/inference.py',
    '/kaggle/working/model_assets/dog_disease_model.pth',
    '/kaggle/working/model_assets/label_encoder.pkl',
    '/kaggle/working/model_assets/model_config.json',
    '/kaggle/working/model_assets/class_names.json'
]

# essential_files directory 
os.makedirs('/kaggle/working/essential_files', exist_ok=True)
for file_path in essential_files:
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)
        shutil.copy(file_path, f'/kaggle/working/essential_files/{file_name}')

shutil.make_archive('/kaggle/working/essential_files', 'zip', '/kaggle/working/essential_files')

print("Zip files created successfully")

Zip files created successfully


# NLP Model Demonstration


### Realistic Data Generation

In [43]:
def create_realistic_symptom_text(row):
    disease_name = row['disease_name'].lower()
    
    # realistic descriptions
    if 'ear infection' in disease_name:
        options = [
            "my dog wont stop shaking his head and his ears smell bad",
            "ears are red and have brown stuff inside, crying when touched",
            "scratching at ears constantly, seems uncomfortable"
        ]
    elif 'allerg' in disease_name:
        options = [
            "licking paws all the time and has red skin on belly",
            "really itchy especially after going outside, getting bald spots",
            "hot spots and watery eyes, seems miserable"
        ]
    elif 'dental' in disease_name:
        options = [
            "really bad breath and yellow teeth, dropping food",
            "pawing at mouth and drooling, wont eat hard food",
            "gums look red and swollen, crying when eating"
        ]
    elif 'giardia' in disease_name:
        options = [
            "watery diarrhea for days, vomiting, losing weight",
            "mucus in stool, lethargic, not eating much",
            "bad diarrhea that wont stop, seems dehydrated"
        ]
    else:
        options = [
            "not acting like himself, just lies around",
            "seems sick and tired, not interested in anything",
            "not eating properly, behavior changed"
        ]
    
    # variations and errors like real users
    complaint = np.random.choice(options)
    
    # realistic duration descriptions
    durations = [
        "for the last couple days",
        "since yesterday", 
        "started a few hours ago",
        "been going on for a while"
    ]
    duration = np.random.choice(durations)
    
    # breed/age/sex mentions
    demo_parts = []
    if pd.notna(row['Breed']):
        if np.random.random() < 0.7:  
            demo_parts.append(f"my {row['Breed']}")
    if pd.notna(row['Estimated_Age_Years_at_HLES']):
        if np.random.random() < 0.5:  
            demo_parts.append(f"is {int(row['Estimated_Age_Years_at_HLES'])} years old")
    if pd.notna(row['Sex_Class_at_HLES']):
        if np.random.random() < 0.4:  
            demo_parts.append(row['Sex_Class_at_HLES'].lower())
    
    # combine parts naturally
    if demo_parts:
        demo_text = " ".join(demo_parts)
        templates = [
            f"{complaint} {duration}. {demo_text}.",
            f"{demo_text.capitalize()}. {complaint} {duration}.",
            f"Im worried about {demo_text}. {complaint} {duration}."
        ]
    else:
        templates = [f"{complaint} {duration}."]
    
    # add typos/errors
    final_text = np.random.choice(templates)
    if np.random.random() < 0.2: 
        errors = {
            'licking': 'lickin', 'scratching': 'scratchin', 'really': 'reallly',
            'vomiting': 'throwin up', 'diarrhea': 'the runs', 'lethargic': 'tired'
        }
        for error, correction in errors.items():
            if error in final_text:
                final_text = final_text.replace(error, correction)
    
    return final_text

### NLP Preprocessing System


In [46]:
# create dataset
df_realistic = df_filtered.copy()
df_realistic['realistic_symptoms'] = df_realistic.apply(create_realistic_symptom_text, axis=1)

print("Realistic Dataset Created:")

# emergency keyword detection
EMERGENCY_KEYWORDS = {
    "seizure", "unconscious", "collapse", "bleeding", "bloat", 
    "distended", "difficulty breathing", "pale gums", "paralysis"
}

def preprocess_user_input(user_input):
    user_input_lower = user_input.lower()
    
    # check for emergencies first
    for keyword in EMERGENCY_KEYWORDS:
        if keyword in user_input_lower:
            return None, {"emergency": True, "keyword": keyword}
    
    # simple synonym standardization
    synonym_map = {
        "puking": "vomiting", "throw up": "vomiting", "barf": "vomiting",
        "the runs": "diarrhea", "loose stool": "diarrhea",
        "lickin": "licking", "scratchin": "scratching",
        "tired": "lethargic", "no energy": "lethargic"
    }
    
    processed_text = user_input_lower
    for slang, standard in synonym_map.items():
        if slang in processed_text:
            processed_text = processed_text.replace(slang, standard)
    
    return processed_text, {"emergency": False}

Realistic Dataset Created:


### Realistic Data Training


In [48]:
# datasets with realistic data
realistic_train_dataset = DogDiseaseDataset(
    df_realistic[df_realistic['dog_id'].isin(train_dogs)]['realistic_symptoms'].tolist(),
    df_realistic[df_realistic['dog_id'].isin(train_dogs)]['disease_label'].tolist(),
    tokenizer
)

realistic_val_dataset = DogDiseaseDataset(
    df_realistic[df_realistic['dog_id'].isin(val_dogs)]['realistic_symptoms'].tolist(),
    df_realistic[df_realistic['dog_id'].isin(val_dogs)]['disease_label'].tolist(),
    tokenizer
)

realistic_test_dataset = DogDiseaseDataset(
    df_realistic[df_realistic['dog_id'].isin(test_dogs)]['realistic_symptoms'].tolist(),
    df_realistic[df_realistic['dog_id'].isin(test_dogs)]['disease_label'].tolist(),
    tokenizer
)

# create data loaders
realistic_train_loader = DataLoader(realistic_train_dataset, batch_size=16, shuffle=True)
realistic_val_loader = DataLoader(realistic_val_dataset, batch_size=16, shuffle=False)
realistic_test_loader = DataLoader(realistic_test_dataset, batch_size=16, shuffle=False)

print("Data Loaders Created:")
print(f"Training batches: {len(realistic_train_loader)}")
print(f"Validation batches: {len(realistic_val_loader)}")
print(f"Test batches: {len(realistic_test_loader)}")

Data Loaders Created:
Training batches: 2276
Validation batches: 479
Test batches: 487


### Model Training


In [50]:
# initialize the model
realistic_model = EnhancedDiseaseClassifier(num_classes=num_classes)
realistic_model = realistic_model.to(device)

# Set up optimizer for this model
realistic_optimizer = AdamW(realistic_model.parameters(), lr=2e-5, weight_decay=0.01)

# train with realistic data 
print("Training Model on Realistic Data")
realistic_train_losses = []
realistic_val_losses = []
realistic_train_accuracies = []
realistic_val_accuracies = []

for epoch in range(3):
    print(f"\nEpoch {epoch + 1}/3")
    print("-" * 30)
    
    # train
    realistic_model.train()
    total_loss = 0
    correct_predictions = 0
    
    for batch in realistic_train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        # forward pass
        loss, logits = realistic_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        
        # backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(realistic_model.parameters(), max_norm=1.0)
        realistic_optimizer.step()
        realistic_optimizer.zero_grad()
        
        total_loss += loss.item()
        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
    
    train_loss = total_loss / len(realistic_train_loader)
    train_acc = correct_predictions.double() / len(realistic_train_loader.dataset)
    realistic_train_losses.append(train_loss)
    realistic_train_accuracies.append(train_acc)
    
    # validate
    realistic_model.eval()
    val_loss, val_acc, _, _ = eval_model(realistic_model, realistic_val_loader, device)
    realistic_val_losses.append(val_loss)
    realistic_val_accuracies.append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# evaluate model
realistic_test_loss, realistic_test_acc, realistic_preds, realistic_labels = eval_model(
    realistic_model, realistic_test_loader, device
)

print("REALISTIC MODEL RESULTS:")
print("=" * 50)
print(f"Test Accuracy: {realistic_test_acc:.4f} ({realistic_test_acc*100:.1f}%)")
print(f"Test Loss: {realistic_test_loss:.4f}")

Training Model on Realistic Data

Epoch 1/3
------------------------------
Train Loss: 1.9281, Train Acc: 0.3871
Val Loss: 1.7939, Val Acc: 0.4203

Epoch 2/3
------------------------------
Train Loss: 1.8106, Train Acc: 0.4138
Val Loss: 1.7681, Val Acc: 0.4243

Epoch 3/3
------------------------------
Train Loss: 1.7935, Train Acc: 0.4161
Val Loss: 1.7685, Val Acc: 0.4252
REALISTIC MODEL RESULTS:
Test Accuracy: 0.4224 (42.2%)
Test Loss: 1.7771
